In [1]:
import numpy as np
from Coursework.ANNModel import model as mod
from Coursework.ANNModel import data as d
from Coursework.PSO import pswarm as ps
from Coursework.PSO import psobehaviour as psh
from Coursework.PSO import history as hs
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

data = d.Data("./Data/1in_linear.txt")
X = data.get_rows()
y = data.get_output()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

D:\GitProjects\Bio-comp\Coursework\ANNModel\model.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#train the model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()

model_history = hs.PSOHistory(model)

Model Compiled!


In [3]:
Tanh_config = [95.63656981, 4.84026189,  1.49822568,  0.8405314,   0.78303066,  0.85137874,  0.85560595]
Cubic_config = [87.89778055, 6.49863902, 1.75988205, 1.49289439, 1.01223157, 1.59668149, 1.2725761] #Fitness: 1248
Linear_config = [6.19884153e+01, 6.19329219e+00, 2.82994987e-02, 9.33192720e-01, 9.06362957e-01, 1.09817852e+00, 1.05772006e+00] # Fitness: 552335863302
Sine_config = [59.35248674, 5.71898616, 1.38796748, 1.48949903, 0.76230353, 0.37289024, 0.98682079] # Fitness: 105

In [4]:
pso = ps.PSO(swarm_size=100, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': 400}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6)
pso = pso.decode_vec(Linear_config)

print(pso.swarm_size)

62


In [5]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)
fitness = pso.run()
print(fitness.location)
print(fitness.fitness)

[ 3.81363922  0.12636214 -0.80035322 -0.07985995 -0.43622693 -0.10903056
  0.39855892  0.16797919 -0.67822984  2.22437568 -0.12550115  0.06135219
 -0.62034881  0.65353133  0.48367088]
1140.1662347120403


In [6]:
pso_best = pso.best.location
model.decode_vec(pso_best)

In [7]:
#model.layers[-1].bias = np.array([0.0])
#model.layers[-1].weights = np.array([[1.0]])


In [8]:
#run model
model.set_training_input(X_test, y_test)
model.one_pass()


In [9]:
model.y_hat

array([-0.54774959, -0.23550251, -0.32142621, -0.12729112,  0.10979465,
        0.42380429,  0.65641075, -0.21389988,  0.80417146,  0.32087862,
       -0.88000275,  0.78945122, -0.65895757,  0.48428958,  0.88136934,
        0.403408  ,  0.04550395, -0.81200728, -0.4474659 , -0.83676547,
       -0.34272742,  0.30004355, -0.25706726, -0.62322625,  0.00245702,
        0.90249761, -0.82464616,  0.50418111,  0.08840509, -0.1706186 ,
       -0.56707237,  0.60101142,  0.17369198])

In [10]:
#score function
correctly_classified = 0


#get predictions
predictions = np.round(model.y_hat, 4)
print(model.y)
print(predictions)

#loop through predictions
for index, pred in enumerate(predictions):

    if np.isclose(y_test[index], pred):
        correctly_classified += 1

print('A,B: ', correctly_classified , " out of ", len(y_test))
correctly_classified = 0
#loop through predictions
for index, pred in enumerate(predictions):
    if np.isclose(pred, y_test[index]):
        correctly_classified += 1

print('B,A: ',correctly_classified , " out of ", len(y_test))

[-0.52 -0.22 -0.3  -0.12  0.1   0.4   0.64 -0.2   0.82  0.3  -0.96  0.8
 -0.64  0.46  0.94  0.38  0.04 -0.84 -0.42 -0.88 -0.32  0.28 -0.24 -0.6
  0.    0.98 -0.86  0.48  0.08 -0.16 -0.54  0.58  0.16]
[-0.5477 -0.2355 -0.3214 -0.1273  0.1098  0.4238  0.6564 -0.2139  0.8042
  0.3209 -0.88    0.7895 -0.659   0.4843  0.8814  0.4034  0.0455 -0.812
 -0.4475 -0.8368 -0.3427  0.3    -0.2571 -0.6232  0.0025  0.9025 -0.8246
  0.5042  0.0884 -0.1706 -0.5671  0.601   0.1737]
A,B:  0  out of  33
B,A:  0  out of  33


In [11]:
print('activation: ', model.layers[-1].activation)
print('bias: ', model.layers[-1].bias)
print('weights: ', model.layers[-1].weights)

activation:  ActivationFunction.HYPERBOLIC_TANGENT
bias:  [-0.12550115]
weights:  [[ 0.06135219]
 [-0.62034881]
 [ 0.65353133]
 [ 0.48367088]]


In [12]:
all_fitness = model_history.historical_particle_fitness()
all_locations = model_history.historical_particle_location()

In [13]:
len(model_history.vec_history[::10])

2480

In [14]:
len(model_history.particle_location[0][::10])

248

In [15]:
#Linear Most fit model: 552335863302
Linear_ann_params = np.array([0.37873092, -0.77783744, -0.15640582,  0.05812352,  0.38093837, -0.51612184,  0.91428793,  0.19934679, 0.67594604,  0.06668804, -0.35705256, -0.36102463,  0.58055201, -0.14048983,  0.45992348])

#Sine Most fit model: 329
Sine_ann_params = np.array([ 3.50680037,  0.2952162 ,  0.71664711, -0.95665899, -0.9055195, -0.27101866,  0.41973435,  0.36889121, -0.28896977,  3.3595307, 0.42623047,  0.30804742,  0.62065839, -0.73504326,  0.70304275])


In [16]:

model.decode_vec(Linear_ann_params)
model.one_pass()

In [19]:
#score function
correctly_classified = 0


#get predictions
predictions = np.round(model.y_hat, 2)
#predictions = model.y_hat
print(model.y)
print(predictions)

#loop through predictions
for index, pred in enumerate(predictions):

    if np.isclose(y_test[index], pred):
        correctly_classified += 1

print('A,B: ', correctly_classified , " out of ", len(y_test))
correctly_classified = 0
#loop through predictions
for index, pred in enumerate(predictions):
    if np.isclose(pred, y_test[index]):
        correctly_classified += 1

print('B,A: ',correctly_classified , " out of ", len(y_test))

[-0.52 -0.22 -0.3  -0.12  0.1   0.4   0.64 -0.2   0.82  0.3  -0.96  0.8
 -0.64  0.46  0.94  0.38  0.04 -0.84 -0.42 -0.88 -0.32  0.28 -0.24 -0.6
  0.    0.98 -0.86  0.48  0.08 -0.16 -0.54  0.58  0.16]
[-0.52 -0.22 -0.3  -0.12  0.1   0.4   0.64 -0.2   0.82  0.3  -0.96  0.8
 -0.64  0.46  0.94  0.38  0.04 -0.84 -0.42 -0.88 -0.32  0.28 -0.24 -0.6
  0.    0.98 -0.86  0.48  0.08 -0.16 -0.54  0.58  0.16]
A,B:  33  out of  33
B,A:  33  out of  33


In [18]:
1/model.loss

602250723504.3938